In [ ]:
pip install nltk

In [5]:
from tkinter import *
import pandas as pd
import nltk
from os import listdir
from os.path import isfile, join
from random import random

def CrearVentana():
    nuevoRoot = Tk()
    nuevoRoot.title("Grupo 5 - Busqueda de noticias") # Escribir nombre de la ventana
    nuevoRoot.resizable(False, False) # Si se puede escalar a lo ancho y a lo alto respectivamente
    nuevoRoot.iconbitmap("icono256.ico") # Escoger imagen que acompaña al titulo
    return nuevoRoot

def CrearFrame(nuevoColor):
    nuevoFrame = Frame(root) # Asociar a la ventana
    nuevoFrame.config(bg=nuevoColor) #Establecer un color al contenedor   
    # Esta linea siempre tiene que ir al final de la funcion:
    nuevoFrame.pack() # Reescalar el frame cuando se reescale la ventana
    return nuevoFrame

def MostrarNoticia():
    if(valorMostradoRanking.get() != "No hay noticias"):
        ruta = valorMostradoRanking.get()[:len(valorMostradoRanking.get()) - 9]
        f = open(ruta, "r")
        leer = f.read()
        contenedorNoticia = Text(frameResultados, width=50, height=14, padx=10) # Crear el cuadro de texto de varias filas
        contenedorNoticia.insert(END, leer)
        scrollNoticia = Scrollbar(frameResultados, command=contenedorNoticia.yview) # Crear y escalar el scroll, y asociarlo a su cuadro de texto
        scrollNoticia.grid(row=1, column=3, rowspan=2, padx=(0, 20), pady=(0, 20), sticky="nsw") # Ubicar el scroll
        contenedorNoticia.grid(row=1, column=2, rowspan=2, padx=(20, 0), pady=(0, 20)) # Ubicar el cuadro de texto de varias filas
        contenedorNoticia.config(yscrollcommand=scrollNoticia.set) # Para que el scroll muestre a qué altura se encuentra el cuadro de texto asociado
        f.close()

root = CrearVentana()  # Crear frame de la ventana
frameConsulta = CrearFrame("#EEEEEE") # Crear contenedor para un grupo de elementos (como un div de HTML)
frameAjustes = CrearFrame("#EEEEEE") # Crear contenedor para un grupo de elementos (como un div de HTML)
frameResultados = CrearFrame("#EEEEEE") # Crear contenedor para un grupo de elementos (como un div de HTML)

labelConsulta = Label(frameConsulta, text="Consulta:").grid(row=0, column=0, padx=10, pady=20, sticky="e") # Crear y ubicar el texto
cuadroConsulta = Entry(frameConsulta)
cuadroConsulta.grid(row=0, column=1, padx=10, pady=20, sticky="ew") # Crear y ubicar el cuadro de texto

labelTopN = Label(frameAjustes, text="TOP-N:").grid(row=0, column=0, padx=(20, 0), sticky="e") # Crear y ubicar el texto
opcionesN = [2, 3, 4, 5, 6, 7, 8, 9, 10, 20] # Opciones que se mostrarán en el desplegable de TOP-N
valorMostradoN = StringVar(frameAjustes) # Para recoger el dato seleccionado
valorMostradoN.set(opcionesN[3]) # Por defecto se mostrará y se escogerá la opción indicada en el enunciado
desplegableN = OptionMenu(frameAjustes, valorMostradoN, *opcionesN) # Crear el desplegable
desplegableN.grid(row=0, column=1, padx=(0, 20), sticky="w") # Ubicar el desplegable

labelFiltrar = Label(frameAjustes, text="Noticiario:").grid(row=0, column=2, padx=(20, 0), sticky="e") # Crear y ubicar el texto
opcionesFiltrar = ["Todos", "El Mundo", "El Pais", "20 minutos"] # Opciones que se mostrarán en el desplegable de Filtrar
valorMostradoFiltrar = StringVar(frameAjustes) # Para recoger el dato seleccionado
valorMostradoFiltrar.set(opcionesFiltrar[0]) # Por defecto se mostrará y se escogerá la opción indicada en el enunciado
desplegableFiltrar = OptionMenu(frameAjustes, valorMostradoFiltrar, *opcionesFiltrar) # Crear el desplegable
desplegableFiltrar.grid(row=0, column=3, padx=(0, 20), sticky="w") # Ubicar el desplegable

def VaciarContenedorNoticia():
    contenedorNoticia = Text(frameResultados, width=50, height=14, padx=10) # Crear el cuadro de texto de varias filas
    contenedorNoticia.insert(END, "")
    scrollNoticia = Scrollbar(frameResultados, command=contenedorNoticia.yview) # Crear y escalar el scroll, y asociarlo a su cuadro de texto
    scrollNoticia.grid(row=1, column=3, rowspan=2, padx=(0, 20), pady=(0, 20), sticky="nsw") # Ubicar el scroll
    contenedorNoticia.grid(row=1, column=2, rowspan=2, padx=(20, 0), pady=(0, 20)) # Ubicar el cuadro de texto de varias filas
    contenedorNoticia.config(yscrollcommand=scrollNoticia.set) # Para que el scroll muestre a qué altura se encuentra el cuadro de texto asociado

def DataFrameWebscraping():
    df = pd.DataFrame()
    medios = ["20Minutos", "ElMundo", "ElPais"]
    categorias = ["Ciencia", "Salud", "Tecnologia"]
    for m in medios:
        for c in categorias:
            rutaBase = m+"/"+c+"/"
            noticias = [f for f in listdir(rutaBase) if isfile(join(rutaBase, f))]
            for n in noticias:
                try:
                    ruta = rutaBase+n
                    texto = open(ruta, "r").read()
                    texto = texto.replace("############################################", "").replace("\n", "")
                    fila = {'Medio': m, 'Categoria': c, 'Texto': texto, 'Ruta': ruta, 'Similitud': random()}
                    df = df.append(fila, ignore_index=True)
                except:
                    a = 0
    df = df.sort_values(by='Similitud', ascending=False)
    df = df.reset_index()
    return df

def BuscarNoticias():
    df_web = DataFrameWebscraping()
    VaciarContenedorNoticia()
    # Buscar las noticias que cumplan los parametros
    indexes = list()
    if(cuadroConsulta.get()!=""): # La formulas pueden dar errores si la consulta tiene 0 palabras
        try:
            if(valorMostradoFiltrar.get()=="Todos"):
                for i in range(len(df_web)):
                    texto = "" + df_web["Ruta"][i] + ' (' + str('%.2f' % round(df_web["Similitud"][i]*100, 3)) + '%)'
                    indexes.append(texto)
            elif(valorMostradoFiltrar.get()=="El Mundo"):    
                for i in range(len(df_web)):
                    if(df_web["Medio"][i]=="ElMundo"):
                        texto = "" + df_web["Ruta"][i] + ' (' + str('%.2f' % round(df_web["Similitud"][i]*100, 3)) + '%)'
                        indexes.append(texto)
            elif(valorMostradoFiltrar.get()=="El Pais"):    
                for i in range(len(df_web)):
                    if(df_web["Medio"][i]=="ElPais"):
                        texto = "" + df_web["Ruta"][i] + ' (' + str('%.2f' % round(df_web["Similitud"][i]*100, 3)) + '%)'
                        indexes.append(texto)
            elif(valorMostradoFiltrar.get()=="20 minutos"):    
                for i in range(len(df_web)):
                    if(df_web["Medio"][i]=="20Minutos"):
                        texto = "" + df_web["Ruta"][i] + ' (' + str('%.2f' % round(df_web["Similitud"][i]*100, 3)) + '%)'
                        indexes.append(texto)
            rutasElegidas = indexes[0:int(valorMostradoN.get())]
            opcionesRanking = rutasElegidas
            if(len(indexes) == 0):
                listaVacia = list()
                listaVacia.append("No hay noticias")
                valorMostradoRanking.set("No hay noticias") # Por defecto se mostrará y se escogerá la opción indicada en el enunciado
                desplegableRanking = OptionMenu(frameResultados, valorMostradoRanking, *listaVacia) # Crear el desplegable
                desplegableRanking.grid(row=1, column=1, padx=(0, 20), pady=(20, 10), sticky="sw") # Ubicar el desplegable
            else:
                valorMostradoRanking.set(indexes[0]) # Por defecto se mostrará y se escogerá la opción indicada en el enunciado
                desplegableRanking = OptionMenu(frameResultados, valorMostradoRanking, *rutasElegidas) # Crear el desplegable
                desplegableRanking.grid(row=1, column=1, padx=(0, 20), pady=(20, 10), sticky="sw") # Ubicar el desplegable
        except:
            rutasElegidas = {"No hay noticias"}
            valorMostradoRanking.set("No hay noticias") # Por defecto se mostrará y se escogerá la opción indicada en el enunciado
            desplegableRanking = OptionMenu(frameResultados, valorMostradoRanking, *indexes) # Crear el desplegable
            desplegableRanking.grid(row=1, column=1, padx=(0, 20), pady=(20, 10), sticky="sw") # Ubicar el desplegable

botonBuscar = Button(frameAjustes, text="Buscar", command=BuscarNoticias).grid(row=0, column=4, padx=20, pady=10) # Crear y ubicar el botón

labelRanking = Label(frameResultados, text="Ranking:").grid(row=1, column=0, padx=(20, 0), pady=(20, 10), sticky="se") # Crear y ubicar el texto
opcionesRanking = ["No hay noticias"] # Opciones que se mostrarán en el desplegable del ranking
valorMostradoRanking = StringVar(frameResultados) # Para recoger el dato seleccionado
valorMostradoRanking.set(opcionesRanking[0]) # Por defecto se mostrará y se escogerá la opción indicada en el enunciado
desplegableRanking = OptionMenu(frameResultados, valorMostradoRanking, *opcionesRanking) # Crear el desplegable
desplegableRanking.grid(row=1, column=1, padx=(0, 20), pady=(20, 10), sticky="sw") # Ubicar el desplegable

botonLeer = Button(frameResultados, text="Leer noticia", command=MostrarNoticia).grid(row=2, column=0, columnspan=2, padx=20, pady=10, sticky="new") # Crear y ubicar el botón

labelNoticia = Label(frameResultados, text="Texto de la noticia:").grid(row=0, column=2, sticky="w", padx=20, pady=(20, 0)) # Crear y ubicar el texto
contenedorNoticia = Text(frameResultados, width=50, height=14, padx=10) # Crear el cuadro de texto de varias filas
scrollNoticia = Scrollbar(frameResultados, command=contenedorNoticia.yview) # Crear y escalar el scroll, y asociarlo a su cuadro de texto
scrollNoticia.grid(row=1, column=3, rowspan=2, padx=(0, 20), pady=(0, 20), sticky="nsw") # Ubicar el scroll
contenedorNoticia.grid(row=1, column=2, rowspan=2, padx=(20, 0), pady=(0, 20)) # Ubicar el cuadro de texto de varias filas
contenedorNoticia.config(yscrollcommand=scrollNoticia.set) # Para que el scroll muestre a qué altura se encuentra el cuadro de texto asociado

root.mainloop() # Bucle infinito para escuchar acciones del usuario (siempre tiene que ir al final)